In [ ]:
# Libraries
import pandas as pd
pd.set_option('display.max_columns', 40)
pd.set_option('display.width', 2000)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.options.mode.chained_assignment = None
%pip install \
--trusted-host 192.168.210.244 \
--index-url http://192.168.210.244:8081/repository/pypi/simple/ \
snowflake-connector-python

In [2]:
import json, snowflake.connector

# establish the connection to snowflake
ctx = snowflake.connector.connect( 
    **json.load(open('/opt/ich/python-snowflake-defaults.json')))
    
# verify and test if connection is working
try: 
    cs = ctx.cursor() 
    cs.execute('SELECT current_version(), current_role(), current_warehouse()')
    print(cs.fetchone())
finally: 
    cs.close()

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/snowflake/connector/options.py:96: UserWarning: You have an incompatible version of 'pyarrow' installed (6.0.0), please install a version that adheres to: 'pyarrow<8.1.0,>=8.0.0; extra == "pandas"'
  warn_incompatible_dep(


Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
We were unable to open a browser window for you, please open the following url manually then paste the URL you are redirected to into the terminal.
URL: https://login.microsoftonline.com/be00e2c6-806c-45f8-84c3-3fc99f64b8d3/saml2?SAMLRequest=pZPRcuIgFIZfJcNeJyHEqmGMHavbWTutdart7nqH5ESpCaRAjO7TL8Y6071ob%2FaOge%2FAx%2FlhcH0oC28P2gglUxQFGHkgucqE3KToeXnr95FnLJMZK5SEFB3BoOvhwLCyqOiotlv5BG81GOu5jaSh7UKKai2pYkYYKlkJhlpOF6OHe0oCTCutrOKqQB9Kvq5gxoC2zvBSkhnh9LbWVjQMm6YJmjhQehMSjHGIk9BRJ%2BTbhT%2B4O33CRyHunHhHOHz%2B7nYj5LkFX2mtz5ChP5bLuT9%2FXCyRN7qojpU0dQl6AXovODw%2F3Z8FjDOodCdJMAmk0nbrQ61VBUGlxZ5ZKITcBUaqJi%2FYDrgqq9q6QwI3CnPIwkJthOvDdJKiaiey2dvtrx7Zfp8dyGPMzHFl8eqmuXtZrK%2F%2BTO7GPyPy27zmD6%2FFruHIe7kETU5BT42pYSpP8Vo3hUnsY%2BLj3jLq0bhLSS%2FoJGSFvImLV0hm28rLHVqPoBRc

('7.4.4', 'ICHT_RG_COVOAM_22016_PROD', 'ICHT_WH_COVOAM_22016_PROD')


True

## Data

In [5]:
# Import 
query = '''
SELECT * from ICHT_PROD.ICHT_COVID.PROBLEM
'''
cur = ctx.cursor().execute(query)
problem_df = pd.DataFrame.from_records(iter(cur), columns=[x[0] for x in cur.description])

In [6]:
path = r'data/international_reciprocal_snomed_distance.csv'
snomed_distance = pd.read_csv(path, index_col=0)

## Analysis

In [7]:
# Get list of snomed codes 
snomed_list = snomed_distance.columns.to_list()

In [9]:
# Filter columns
problem_df = problem_df[['SUBJECT', 'PROBLEM', 'PROBLEM_DT_TM']]

In [10]:
# Filter for snomed codes in list
problem_df = problem_df[problem_df['PROBLEM'].isin(snomed_list)]

In [32]:
# Get list of subjects
subject_list = problem_df.SUBJECT.unique().tolist()

In [ ]:
new_df = pd.DataFrame(columns=['new_subject', 'SUBJECT', 'PROBLEM'])
n = -1
for subject in subject_list:
    temp_df = problem_df[problem_df['SUBJECT'] == subject]
    temp_df.sort_values(by='PROBLEM_DT_TM', inplace = True)
    for x in temp_df['PROBLEM_DT_TM'].unique():
        n += 1
        temp_df2 = temp_df[temp_df['PROBLEM_DT_TM'] <= x]
        temp_df2['new_subject'] = n
        new_df = pd.concat([new_df, temp_df2], ignore_index=True)

In [ ]:
final_dummies = pd.DataFrame()
for n in range(0, 95157):
    temp_df = new_df[new_df['new_subject'] == n]
        
    temp_dummies = pd.get_dummies(temp_df, columns=['PROBLEM'])

    temp_df_2 = temp_dummies.copy()
    temp_df_2 = pd.DataFrame(temp_dummies.iloc[-1,:3]).transpose()
    temp_df_2.reset_index(inplace=True, drop=True)

    temp_dummies2 = pd.DataFrame(temp_dummies.iloc[:,3:].sum()).transpose()

    temp_df_2 = temp_df_2.join(temp_dummies2)
    
    final_dummies = pd.concat([final_dummies, temp_df_2], ignore_index=True)

    if (n / 1000).is_integer():
        print(n)
        # Load
        path = r'working_problem_dummies.csv'
        working_dummies = pd.read_csv(path)
        # Concat
        working_dummies = pd.concat([working_dummies, final_dummies], ignore_index=True)
        # Save
        #working_dummies.to_csv('working_problem_dummies.csv', index=False)
        # Delete
        del working_dummies
        del final_dummies
        final_dummies = pd.DataFrame()
        print('new 1000')

In [ ]:
# Load
path = r'working_problem_dummies.csv'
working_dummies = pd.read_csv(path)
# Concat
working_dummies_2 = pd.concat([working_dummies, final_dummies], ignore_index=True)
# Save
#working_dummies_2.to_csv('final_working_problem_dummies.csv', index=False)

In [ ]:
# Fill na ith 0
working_dummies_2.fillna(0, inplace=True)

In [ ]:
# Set max value to 1
problem_dummies_3 = working_dummies_2.iloc[:,3:]
problem_dummies_3.where(problem_dummies_3 <=1, 1, inplace=True)
problem_dummies_4 = working_dummies_2.iloc[:,:3].join(problem_dummies_3)

In [ ]:
# Save
#problem_dummies_4.to_csv('final_problem_dummies.csv', index=False)